# Backtest Version 1 

Uses pre-downloaded data on historicals to perform backtesting to now. 

Assumptions: 4% gain, on a $100 USD per tick loss. 


In [18]:
import QuantLib as ql 
from src.kata_alpaca_engine.engine_utilities import BackTestData
from src.kata_models.GaussianProcessModel import RegressionModel
from src.kata_alpaca_engine.ingestion_engine import Redis, RedisTableUtility
import plotly.graph_objects as go
from datetime import datetime



In [19]:
connection  = Redis(host='red', port='6379')
red_table = RedisTableUtility(connection)

In [20]:
red_table.get_tags('_NVDA_HISTORICAL')

['2024-07-26', '2024-07-29', '2024-08-01']

In [21]:
df = red_table.get(key='_NVDA_HISTORICAL', date_key='2024-08-01')

In [78]:
btest = BackTestData(df)

In [113]:
dataset_index = -1

start_date = datetime(year=2023, month=10, day=29)
end_date = datetime(year=2024, month=2, day=5)
projection_date = ql.Date(9, 2, 2024)

btest.create_backtest_data(
   start_date,
   end_date, 
   projection_date,
   store_prices=True
)


btest.fit_model(dataset_index, lambda x,y: RegressionModel(x,y))


yfit, ysigma = btest.predict(dataset_index) 
test_yfit , lower, upper = btest.create_bands(yfit, ysigma)
training_f, lower_f, upper_f = btest.training_bands(dataset_index)



prediction_markers = go.Scatter(x=btest.prediction_data[dataset_index], 
                            y=btest.prediction_data_price[dataset_index], 
                            mode='markers',
                            marker=dict(color='red'),
                            name='test_data')


xt = btest.prediction_data[dataset_index]
bands = [
go.Scatter(x=xt, y=test_yfit, mode='lines', line=dict(color='black', width=2), name='test_fit'),
go.Scatter(x=xt, y=upper, mode = 'lines', line=dict(color='black', width=2), name='Upper Band'),
go.Scatter(x=xt, y=lower, mode = 'lines', line=dict(color='black', width=2), name='Lower Band'),
go.Scatter(x=xt, y=upper, fill=None, mode='lines', line=dict(color='rgba(0,100,80,0.2)'),  name='Upper Band', showlegend=False),
go.Scatter(x=xt, y=lower, fill='tonexty', mode='lines', line=dict(color='rgba(0,100,80,0.2)'), name='Lower Band', showlegend=False),
]

x = btest.data[dataset_index][0]
training_fit_bands = [
go.Scatter(x=x, y=training_f, name='training_fit', mode='lines'),
go.Scatter(x=x, y=upper_f, line=dict(color='black', width=2), name='Upper Band'),
go.Scatter(x=x, y=lower_f, line=dict(color='black', width=2), name='Lower Band'),
go.Scatter(x=x, y=upper_f, fill=None, mode='lines', line=dict(color='rgba(0,100,80,0.2)'), showlegend=False),
go.Scatter(x=x, y=lower_f, fill='tonexty', mode='lines', line=dict(color='rgba(0,100,80,0.2)'), showlegend=False),
] 

figure_inputs = [prediction_markers, *bands]

fig = go.Figure(figure_inputs)

fig.show()

/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [115]:
from datetime import datetime, timedelta



def shifted_dates(start_date_str, end_date_str, projection_day='Friday'):
    # Convert input date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    # Shift dates by 7 days
    new_start_date = start_date + timedelta(days=7)
    new_end_date = end_date + timedelta(days=7)

    # Find the next specified day (Friday) after the new start date
    day_of_week = new_start_date.weekday()
    days_to_projection = (4 - day_of_week) % 7  # 4 is Friday
    new_projection_date = new_end_date + timedelta(days=days_to_projection)

    # Format dates to string
    new_start_date_str = new_start_date
    new_end_date_str = new_end_date
    new_projection_date_str = new_projection_date

    return new_start_date_str, new_end_date_str, new_projection_date_str



In [116]:
date_format = '%Y-%m-%d'

for i in range(14):
    new_start_date, new_end_date, new_projected_date = shifted_dates(
        start_date.strftime(date_format),
        end_date.strftime(date_format),
        btest.convert_ql_to_datetime(projection_date).strftime(date_format),
    )


    new_projected_date = btest.convert_datetime_to_ql(new_projected_date)

    dataset_index = -1
    btest.create_backtest_data(
        new_start_date,
        new_end_date, 
        new_projected_date, 
    store_prices=True
    )

    start_date = new_start_date
    end_date = new_end_date
    projection_date = new_projected_date

    btest.fit_model(dataset_index, lambda x,y: RegressionModel(x,y))


    yfit, ysigma = btest.predict(dataset_index) 
    test_yfit , lower, upper = btest.create_bands(yfit, ysigma)
    training_f, lower_f, upper_f = btest.training_bands(dataset_index)



    prediction_markers = go.Scatter(x=btest.prediction_data[dataset_index], 
                                y=btest.prediction_data_price[dataset_index], 
                                mode='markers',
                                marker=dict(color='red'),
                                name='test_data')


    xt = btest.prediction_data[dataset_index]

    bands = [
    go.Scatter(x=xt, y=test_yfit, mode='lines', line=dict(color='black', width=2), name='test_fit'),
    go.Scatter(x=xt, y=upper, mode = 'lines', line=dict(color='black', width=2), name='Upper Band'),
    go.Scatter(x=xt, y=lower, mode = 'lines', line=dict(color='black', width=2), name='Lower Band'),
    go.Scatter(x=xt, y=upper, fill=None, mode='lines', line=dict(color='rgba(0,100,80,0.2)'),  name='Upper Band', showlegend=False),
    go.Scatter(x=xt, y=lower, fill='tonexty', mode='lines', line=dict(color='rgba(0,100,80,0.2)'), name='Lower Band', showlegend=False),
    ]

    x = btest.data[dataset_index][0]
    training_fit_bands = [
    go.Scatter(x=x, y=training_f, name='training_fit', mode='lines'),
    go.Scatter(x=x, y=upper_f, line=dict(color='black', width=2), name='Upper Band'),
    go.Scatter(x=x, y=lower_f, line=dict(color='black', width=2), name='Lower Band'),
    go.Scatter(x=x, y=upper_f, fill=None, mode='lines', line=dict(color='rgba(0,100,80,0.2)'), showlegend=False),
    go.Scatter(x=x, y=lower_f, fill='tonexty', mode='lines', line=dict(color='rgba(0,100,80,0.2)'), showlegend=False),
    ] 

    figure_inputs += [prediction_markers, *bands]

    fig = go.Figure(figure_inputs)

/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k2__k2__sigma_0 is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The o

In [117]:
fig.show()